# 테스트 전 토큰화 함수 및 임폴트

In [1]:
import pandas as pd  # ✅ pandas 누락
from kiwipiepy import Kiwi  # ✅ kiwi 토크나이저
import joblib  # ✅ 모델 불러오기

# ✅ tokenizer 함수 정의
kiwi = Kiwi()


def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:
            if pos in {"VV", "VA"}:
                word = word + "다"
            tokens.append(word)
    return " ".join(tokens)

# 앙상블 Stacking 테스트

In [3]:
from sklearn.metrics import classification_report

# 테스트셋 불러오기
test_df = pd.read_csv("../../dataset/1차모델_테스트데이터셋.csv")  # 실제 경로로 바꿔줘
test_df["processed_text"] = test_df["text"].astype(str).apply(tokenize_and_filter)

X_test = test_df["processed_text"]
y_test = test_df["is_phishing"]

# 저장된 모델 불러오기
import joblib

model = joblib.load("../../models/pipeline_stacking.pkl")

# 예측 및 성능 확인
y_pred_test = model.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       500
           1       1.00      0.89      0.94       500

    accuracy                           0.94      1000
   macro avg       0.95      0.95      0.94      1000
weighted avg       0.95      0.94      0.94      1000

